In [20]:
#Se importan la librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import quandl
#import pandas_datareader.data as web
import datetime
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Lectura del archivo .csv para manipular la data
f = open('predict_stockMSFT.csv','rU')
df = pd.read_csv(f , parse_dates = True, index_col = 0)  #analiza la columna 1 como fechas
df

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2010-01-04,30.620,31.1000,30.5900,30.950,38409100.0,0.00,1.0,24.568238,24.953370,24.544167,24.833017,38409100.0
2010-01-05,30.850,31.1000,30.6400,30.960,49749600.0,0.00,1.0,24.752781,24.953370,24.584285,24.841040,49749600.0
2010-01-06,30.880,31.0800,30.5200,30.770,58182400.0,0.00,1.0,24.776851,24.937323,24.488002,24.688592,58182400.0
2010-01-07,30.630,30.7000,30.1900,30.452,50559700.0,0.00,1.0,24.576262,24.632427,24.223224,24.433442,50559700.0
2010-01-08,30.280,30.8800,30.2400,30.660,51197400.0,0.00,1.0,24.295436,24.776851,24.263342,24.600332,51197400.0
2010-01-11,30.710,30.7600,30.1200,30.270,68754700.0,0.00,1.0,24.640450,24.680568,24.167058,24.287412,68754700.0
2010-01-12,30.150,30.4000,29.9100,30.070,65912100.0,0.00,1.0,24.191129,24.391719,23.998563,24.126940,65912100.0
2010-01-13,30.260,30.5200,30.0100,30.350,51863500.0,0.00,1.0,24.279389,24.488002,24.078799,24.351601,51863500.0
2010-01-14,30.310,31.1000,30.2600,30.960,63228100.0,0.00,1.0,24.319507,24.953370,24.279389,24.841040,63228100.0


In [21]:
#Calculo el porcentaje del promedio de cambios durante el día
df['Promedio_HL'] = (df['High'] - df['Low']) / df['Low'] * 100.0
df

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,Promedio_HL
Date,,,,,,,,,,,,,
2010-01-04,30.620,31.1000,30.5900,30.950,38409100.0,0.00,1.0,24.568238,24.953370,24.544167,24.833017,38409100.0,1.667212
2010-01-05,30.850,31.1000,30.6400,30.960,49749600.0,0.00,1.0,24.752781,24.953370,24.584285,24.841040,49749600.0,1.501305
2010-01-06,30.880,31.0800,30.5200,30.770,58182400.0,0.00,1.0,24.776851,24.937323,24.488002,24.688592,58182400.0,1.834862
2010-01-07,30.630,30.7000,30.1900,30.452,50559700.0,0.00,1.0,24.576262,24.632427,24.223224,24.433442,50559700.0,1.689301
2010-01-08,30.280,30.8800,30.2400,30.660,51197400.0,0.00,1.0,24.295436,24.776851,24.263342,24.600332,51197400.0,2.116402
2010-01-11,30.710,30.7600,30.1200,30.270,68754700.0,0.00,1.0,24.640450,24.680568,24.167058,24.287412,68754700.0,2.124834
2010-01-12,30.150,30.4000,29.9100,30.070,65912100.0,0.00,1.0,24.191129,24.391719,23.998563,24.126940,65912100.0,1.638248
2010-01-13,30.260,30.5200,30.0100,30.350,51863500.0,0.00,1.0,24.279389,24.488002,24.078799,24.351601,51863500.0,1.699434
2010-01-14,30.310,31.1000,30.2600,30.960,63228100.0,0.00,1.0,24.319507,24.953370,24.279389,24.841040,63228100.0,2.775942


In [22]:
df['Promedio_Cambio_dia'] = (df['Close'] - df['Open']) / df['Open'] * 100.0
df

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,Promedio_HL,Promedio_Cambio_dia
Date,,,,,,,,,,,,,,
2010-01-04,30.620,31.1000,30.5900,30.950,38409100.0,0.00,1.0,24.568238,24.953370,24.544167,24.833017,38409100.0,1.667212,1.077727
2010-01-05,30.850,31.1000,30.6400,30.960,49749600.0,0.00,1.0,24.752781,24.953370,24.584285,24.841040,49749600.0,1.501305,0.356564
2010-01-06,30.880,31.0800,30.5200,30.770,58182400.0,0.00,1.0,24.776851,24.937323,24.488002,24.688592,58182400.0,1.834862,-0.356218
2010-01-07,30.630,30.7000,30.1900,30.452,50559700.0,0.00,1.0,24.576262,24.632427,24.223224,24.433442,50559700.0,1.689301,-0.581130
2010-01-08,30.280,30.8800,30.2400,30.660,51197400.0,0.00,1.0,24.295436,24.776851,24.263342,24.600332,51197400.0,2.116402,1.254954
2010-01-11,30.710,30.7600,30.1200,30.270,68754700.0,0.00,1.0,24.640450,24.680568,24.167058,24.287412,68754700.0,2.124834,-1.432758
2010-01-12,30.150,30.4000,29.9100,30.070,65912100.0,0.00,1.0,24.191129,24.391719,23.998563,24.126940,65912100.0,1.638248,-0.265340
2010-01-13,30.260,30.5200,30.0100,30.350,51863500.0,0.00,1.0,24.279389,24.488002,24.078799,24.351601,51863500.0,1.699434,0.297422
2010-01-14,30.310,31.1000,30.2600,30.960,63228100.0,0.00,1.0,24.319507,24.953370,24.279389,24.841040,63228100.0,2.775942,2.144507


In [24]:
#Selecciono solo los datos que voy a utilizar en el análisis
df = df[['Close', 'Promedio_HL', 'Promedio_Cambio_dia', 'Volume']]
df

,Close,Promedio_HL,Promedio_Cambio_dia,Volume
Date,,,,
2010-01-04,30.950,1.667212,1.077727,38409100.0
2010-01-05,30.960,1.501305,0.356564,49749600.0
2010-01-06,30.770,1.834862,-0.356218,58182400.0
2010-01-07,30.452,1.689301,-0.581130,50559700.0
2010-01-08,30.660,2.116402,1.254954,51197400.0
2010-01-11,30.270,2.124834,-1.432758,68754700.0
2010-01-12,30.070,1.638248,-0.265340,65912100.0
2010-01-13,30.350,1.699434,0.297422,51863500.0
2010-01-14,30.960,2.775942,2.144507,63228100.0


In [25]:
#En caso que alguna data no se encuentre NAN se rellena con el valor 99999
df.fillna(value=-99999, inplace=True)
df

C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Close,Promedio_HL,Promedio_Cambio_dia,Volume
Date,,,,
2010-01-04,30.950,1.667212,1.077727,38409100.0
2010-01-05,30.960,1.501305,0.356564,49749600.0
2010-01-06,30.770,1.834862,-0.356218,58182400.0
2010-01-07,30.452,1.689301,-0.581130,50559700.0
2010-01-08,30.660,2.116402,1.254954,51197400.0
2010-01-11,30.270,2.124834,-1.432758,68754700.0
2010-01-12,30.070,1.638248,-0.265340,65912100.0
2010-01-13,30.350,1.699434,0.297422,51863500.0
2010-01-14,30.960,2.775942,2.144507,63228100.0


In [35]:
#Se define la columna con las etiquetas o respuestas
data_etiquetada = 'Close'
#Se define el número de datos a predecir
pronostico = int(30)
#Se crea una nueva columna con el pronostico de la data
df['Pronostico'] = df[data_etiquetada].shift(-pronostico)
#Se crea el conjunto de datos con las características
X = np.array(df.drop(['Pronostico'], 1))
#Se realiza el escalamiento de los datos 
X = preprocessing.scale(X)
#Se elimina las filas de los datos a predecir
X = X[:-pronostico]
#Se crea una variable que contiene solamente los datos para predecir
X_prediccion = X[-pronostico:]
#Se crea una variable con las etiquetas o respuestas
y = np.array(df['Pronostico'])
#Se elimina las filas de los datos a predecir
y = y[:-pronostico]
#Se selecciona el 20% de la data entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #con 0.3

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
#Tipo de analisis para el pronostico
algoritmo = svm.SVR() #Support Vector Regression.
#Entrenamos el algoritmo
algoritmo.fit(X_train, y_train)
#Calculamos la confianza tomando en cuanta los datos de prueba
confianza = algoritmo.score(X_test, y_test)
print('Confianza = ', confianza)

('Confianza = ', 0.9246677225463291)


In [37]:
#Utilizando el modelo realizamos una predicción
prediccion_manana = algoritmo.predict(X_prediccion)
print(prediccion_manana)

[82.0591925  99.64317057 79.7081616  91.40474828 99.9265574  81.21755517
 87.62968967 95.48960471 91.02044626 96.53819683 92.20787537 95.20760085
 90.52969294 94.4092134  97.82046914 89.56824451 95.74984261 94.88768076
 77.43771338 86.63700841 95.4749481  63.12822207 92.18372717 66.14525927
 82.34581554 98.2954029  96.40555313 70.3666287  96.87695162 93.67518568]


In [38]:
#Tipo de analisis para el pronostico
algoritmo = LinearRegression() #linear regression
#Entrenamos el algoritmo
algoritmo.fit(X_train, y_train)
#Calculamos la confianza tomando en cuanta los datos de prueba
confianza = algoritmo.score(X_test, y_test)
print('Confianza = ', confianza)

('Confianza = ', 0.9872497008334239)


In [39]:
#Utilizando el modelo realizamos una predicción
prediccion_manana = algoritmo.predict(X_prediccion)
print(prediccion_manana)

[102.38752009 102.85665228 101.54233936 102.32304919 102.43435016
 103.75100798 103.06577094 103.57196237 104.99891517 105.7795529
 106.04283855 105.8655277  108.13751809 109.46050174 109.05861175
 110.01845049 109.27965346 108.46050555 110.5157176  112.0801239
 111.97586969 115.06451608 114.02268838 112.25462753 109.70522583
 110.25576847 110.40055811 111.70956033 112.28070337 112.36559227]
